In [100]:
import pdb
import pickle
import string
import sys
import time

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import scipy
import sklearn
from gensim.models import KeyedVectors
from nltk.corpus import stopwords, twitter_samples
from nltk.tokenize import TweetTokenizer

sys.path.append('..')
from utils import cosine_similarity, get_dict, process_tweet

# 1. The word embeddings data for English and French words
Write a program that translates English to French.

- Eng embeddings: https://code.google.com/archive/p/word2vec/ (GoogleNews-vectors-negative300.bin.gz)
- Fr embeddings:
```
curl -o ./wiki.multi.fr.vec https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.fr.vec
```

In [101]:
DATA = '../../../../data'

In [102]:
en_embeddings_subset = pickle.load(
    open(f'{DATA}/en_embeddings.pkl', 'rb'))
fr_embeddings_subset = pickle.load(
    open(f'{DATA}/fr_embeddings.pkl', 'rb'))

In [103]:
type(en_embeddings_subset)

dict

In [104]:
print(len(en_embeddings_subset['the']))
en_embeddings_subset['the'][:10]

300


array([ 0.08007812,  0.10498047,  0.04980469,  0.0534668 , -0.06738281,
       -0.12060547,  0.03515625, -0.11865234,  0.04394531,  0.03015137],
      dtype=float32)

In [105]:
# loading the english to french dictionaries
en_fr_train = get_dict(f'{DATA}/en-fr.train.txt')
print('The length of the English to French training dictionary is', 
      len(en_fr_train))
en_fr_test = get_dict(f'{DATA}/en-fr.test.txt')
print('The length of the English to French test dictionary is', 
      len(en_fr_train))

The length of the English to French training dictionary is 5000
The length of the English to French test dictionary is 5000


In [106]:
type(en_fr_train)

dict

In [107]:
en_fr_train['dog']

'chienne'

## 1.1 Generate embedding and transform matrices
#### Exercise 01: Translating English dictionary to French by using embeddings

implement a function `get_matrices`, which takes the loaded data
and returns matrices `X` and `Y`.

Inputs:
- `en_fr` : English to French dictionary
- `en_embeddings` : English to embeddings dictionary
- `fr_embeddings` : French to embeddings dictionary

Returns:
- Matrix `X` and matrix `Y`, where each row in X is the word embedding for an
english word, and the same row in Y is the word embedding for the French
version of that English word.

Use the `en_fr` dictionary to ensure that the ith row in the `X` matrix
corresponds to the ith row in the `Y` matrix.

In [108]:
def get_matrices(en_fr, french_vecs, english_vecs):
    """
    Input:
        en_fr: English to French dictionary
        french_vecs: French words to their corresponding word embeddings.
        english_vecs: English words to their corresponding word 
          embeddings.
    Output: 
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the projection matrix that minimizes the F norm ||X R -Y||^2.
    """
    X = []
    Y = []
    english_set = set(english_vecs.keys())
    french_set = set(french_vecs.keys())
    french_words = set(en_fr.values())
    for en_word, fr_word in en_fr.items():
        if fr_word in french_set and en_word in english_set:
            en_vec = english_vecs[en_word]
            fr_vec = french_vecs[fr_word]
            X.append(en_vec)
            Y.append(fr_vec)
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

In [109]:
X_train, Y_train = get_matrices(
    en_fr_train, fr_embeddings_subset, en_embeddings_subset)

# 2. Translations

## 2.1 Translation as linear transformation of embeddings

Given dictionaries of English and French word embeddings, create a transformation matrix `R`
* Given an English word embedding, **e**, you can multiply **eR** to get a new word embedding **f**.
    * Both **e** and **f** are [row vectors](https://en.wikipedia.org/wiki/Row_and_column_vectors).
* You can then compute the nearest neighbors to `f` in the french embeddings and recommend the word that is most similar to the transformed word embedding.

### Describing translation as the minimization problem

Find a matrix `R` that minimizes the following equation. 

$$\arg \min _{\mathbf{R}}\| \mathbf{X R} - \mathbf{Y}\|_{F}\tag{1} $$

### Frobenius norm

The Frobenius norm of a matrix $A$ (assuming it is of dimension $m,n$) is defined as the square root of the sum of the absolute squares of its elements:

$$\|\mathbf{A}\|_{F} \equiv \sqrt{\sum_{i=1}^{m} \sum_{j=1}^{n}\left|a_{i j}\right|^{2}}\tag{2}$$

### Actual loss function
In the real world applications, the Frobenius norm loss:

$$\| \mathbf{XR} - \mathbf{Y}\|_{F}$$

is often replaced by it's squared value divided by $m$:

$$ \frac{1}{m} \|  \mathbf{X R} - \mathbf{Y} \|_{F}^{2}$$

where $m$ is the number of examples (rows in $\mathbf{X}$).

* The same R is found when using this loss function versus the original Frobenius norm.
* The reason for taking the square is that it's easier to compute the gradient of the squared Frobenius.
* The reason for dividing by $m$ is that we're more interested in the average loss per embedding than the  loss for the entire training set.
    * The loss for all training set increases with more words (training examples),
    so taking the average helps us to track the average loss regardless of the size of the training set.

#### Step 1: Computing the loss
* The loss function will be squared Frobenoius norm of the difference between
matrix and its approximation, divided by the number of training examples $m$.
* Its formula is:
$$ L(X, Y, R)=\frac{1}{m}\sum_{i=1}^{m} \sum_{j=1}^{n}\left( a_{i j} \right)^{2}$$

where $a_{i j}$ is value in $i$th row and $j$th column of the matrix $\mathbf{XR}-\mathbf{Y}$.

In [110]:
def compute_loss(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m, n) where the columns are the English 
          embeddings.
        Y: a matrix of dimension (m, n) where the columns corresponding to 
          the French embeddings.
        R: a matrix of dimension (n, n) - transformation matrix from 
          English to French vector space embeddings.
    Outputs:
        L: a matrix of dimension (m, n) - the value of the loss function 
          for given X, Y and R.
    '''
    m = X.shape[0]
    err = X @ R - Y
    loss = (err ** 2).sum() / m
    return loss

In [111]:
X = np.array([[0, 1, 2],
              [1, 2, 3],
              [2, 3, 4],
              [3, 4, 5]])
Y = np.array([[0, 1, 3],
              [1, 3, 5],
              [2, 3, 5],
              [3, 5, 7]])
R = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

In [112]:
compute_loss(X, Y, R)

3.0

In [113]:
compute_loss(X, X, R)

0.0

### Step 2: Computing the gradient of loss wrt transform matrix R

* Calculate the gradient of the loss with respect to transform matrix `R`.
* The gradient is a matrix that encodes how much a small change in `R`
affect the change in the loss function.
* The gradient gives us the direction in which we should decrease `R`
to minimize the loss.
* $m$ is the number of training examples (number of rows in $X$).
* The formula for the gradient of the loss function $𝐿(𝑋,𝑌,𝑅)$ is:

$$\frac{d}{dR}𝐿(𝑋,𝑌,𝑅)=\frac{d}{dR}\Big(\frac{1}{m}\| X R -Y\|_{F}^{2}\Big) = \frac{2}{m}X^{T} (X R - Y)$$

In [114]:
def compute_gradient(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m, n) where the columns are the English 
          embeddings.
        Y: a matrix of dimension (m, n) where the columns corresponding to
          the French embeddings.
        R: a matrix of dimension (n, n) - transformation matrix from 
          English to French vector space embeddings.
    Outputs:
        g: a matrix of dimension (n, n) - gradient of the loss function L
          for given X, Y and R.
    '''
    m = X.shape[0]
    gradient = (2/m) * X.T @ (X @ R - Y)
    return gradient

In [115]:
compute_gradient(X, Y, R)

array([[  0.,  -2.,  -5.],
       [  0.,  -3.,  -8.],
       [  0.,  -4., -11.]])

In [116]:
compute_gradient(X, X, R)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

### Step 3: Finding the optimal R with gradient descent algorithm

#### Gradient descent

[Gradient descent](https://ml-cheatsheet.readthedocs.io/en/latest/gradient_descent.html) is an iterative algorithm which is used in searching for the optimum of the function. 
* Earlier, we've mentioned that the gradient of the loss with respect to the matrix encodes how much a tiny change in some coordinate of that matrix affect the change of loss function.
* Gradient descent uses that information to iteratively change matrix `R` until we reach a point where the loss is minimized. 

Pseudocode:
1. Calculate gradient $g$ of the loss with respect to the matrix $R$.
2. Update $R$ with the formula:
$$R_{\text{new}}= R_{\text{old}}-\alpha g$$

Where $\alpha$ is the learning rate, which is a scalar.

In [117]:
def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003):
    '''
    Inputs:
        X: a matrix of dimension (m, n) where the columns are the English 
          embeddings.
        Y: a matrix of dimension (m, n) where the columns corresponding to
          the French embeddings.
        train_steps: positive int - describes how many steps will gradient
          descent algorithm do.
        learning_rate: positive float - describes how big steps will
          gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n, n) - the projection matrix that 
          minimizes the F norm ||XR - Y||^2
    '''
    #np.random.seed(129)

    # the number of columns in X is the number of dimensions for a word vector (e.g. 300)
    # R is a square matrix with length equal to the number of dimensions in th  word embedding
    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if i % 25 == 0:
            print(
                f'loss at iteration {i} is: {compute_loss(X, Y, R):.4f}')
        ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
        # use the function that you defined to compute the gradient
        gradient = compute_gradient(X, Y, R)

        # update R by subtracting the learning rate times gradient
        R -= (learning_rate * gradient)
        ### END CODE HERE ###
    return R

In [118]:
R_train = align_embeddings(
    X_train, Y_train, train_steps=1000, learning_rate=0.8)

loss at iteration 0 is: 950.5817
loss at iteration 25 is: 97.7984
loss at iteration 50 is: 26.9240
loss at iteration 75 is: 9.8540
loss at iteration 100 is: 4.4168
loss at iteration 125 is: 2.3515
loss at iteration 150 is: 1.4622
loss at iteration 175 is: 1.0424
loss at iteration 200 is: 0.8305
loss at iteration 225 is: 0.7178
loss at iteration 250 is: 0.6555
loss at iteration 275 is: 0.6199
loss at iteration 300 is: 0.5990
loss at iteration 325 is: 0.5864
loss at iteration 350 is: 0.5786
loss at iteration 375 is: 0.5738
loss at iteration 400 is: 0.5707
loss at iteration 425 is: 0.5687
loss at iteration 450 is: 0.5674
loss at iteration 475 is: 0.5665
loss at iteration 500 is: 0.5659
loss at iteration 525 is: 0.5655
loss at iteration 550 is: 0.5653
loss at iteration 575 is: 0.5651
loss at iteration 600 is: 0.5650
loss at iteration 625 is: 0.5649
loss at iteration 650 is: 0.5648
loss at iteration 675 is: 0.5648
loss at iteration 700 is: 0.5648
loss at iteration 725 is: 0.5647
loss at ite

## 2.2 Testing the translation

### k-Nearest neighbors algorithm

[k-Nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) 
* k-NN is a method which takes a vector as input and finds the other vectors in the dataset that are closest to it. 
* The 'k' is the number of "nearest neighbors" to find (e.g. k=2 finds the closest two neighbors).

### Searching for the translation embedding
Since we're approximating the translation function from English to French embeddings by a linear transformation matrix $\mathbf{R}$, most of the time we won't get the exact embedding of a French word when we transform embedding $\mathbf{e}$ of some particular English word into the French embedding space. 
* This is where $k$-NN becomes really useful! By using $1$-NN with $\mathbf{eR}$ as input, we can search for an embedding $\mathbf{f}$ (as a row) in the matrix $\mathbf{Y}$ which is the closest to the transformed vector $\mathbf{eR}$

### Cosine similarity
Cosine similarity between vectors $u$ and $v$ calculated as the cosine of the angle between them.
The formula is 

$$\cos(u,v)=\frac{u\cdot v}{\left\|u\right\|\left\|v\right\|}$$

* $\cos(u,v)$ = $1$ when $u$ and $v$ lie on the same line and have the same direction.
* $\cos(u,v)$ is $-1$ when they have exactly opposite directions.
* $\cos(u,v)$ is $0$ when the vectors are orthogonal (perpendicular) to each other.

#### Note: Distance and similarity are pretty much opposite things.
* We can obtain distance metric from cosine similarity, but the cosine similarity can't be used directly as the distance metric. 
* When the cosine similarity increases (towards $1$), the "distance" between the two vectors decreases (towards $0$). 
* We can define the cosine distance between $u$ and $v$ as
$$d_{\text{cos}}(u,v)=1-\cos(u,v)$$

Inputs:
* Vector `v`,
* A set of possible nearest neighbors `candidates`
* `k` nearest neighbors to find.
* The distance metric should be based on cosine similarity.
* `cosine_similarity` function is already implemented and imported for you. It's arguments are two vectors and it returns the cosine of the angle between them.
* Iterate over rows in `candidates`, and save the result of similarities between current row and vector `v` in a python list. Take care that similarities are in the same order as row vectors of `candidates`.
* Now you can use [numpy argsort]( https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html#numpy.argsort) to sort the indices for the rows of `candidates`.

In [119]:
def nearest_neighbor(v, candidates, k=1):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    sims = []
    for row in candidates:
        cos_similarity = cosine_similarity(v, row)
        sims.append(cos_similarity)
    sorted_ids = np.argsort(sims)
    k_idx = sorted_ids[-k:]
    return k_idx

In [120]:
v = np.array([1, 0, 1])
candidates = np.array(
    [[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates[nearest_neighbor(v, candidates, 3)])

[[9 9 9]
 [1 0 5]
 [2 0 1]]


In [121]:
def test_vocabulary(X, Y, R):
    '''
    Input:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the transform matrix which translates word embeddings from
        English to French word vector space.
    Output:
        accuracy: for the English to French capitals
    '''
    pred = X @ R
    n_correct = 0
    for i, vec in enumerate(pred):
        pred_idx = nearest_neighbor(vec, Y)
        if pred_idx == i:
            n_correct += 1
    accuracy = n_correct / len(pred)
    return accuracy

In [122]:
X_val, Y_val = get_matrices(
    en_fr_test, fr_embeddings_subset, en_embeddings_subset)

In [123]:
X = X_val
Y = Y_val
R = R_train

pred = X @ R
n_correct = 0
nearest_neighbor(pred[0, :], Y, 1)

array([0])

In [124]:
acc = test_vocabulary(X_val, Y_val, R_train)
print(f"accuracy on test set is {acc:.3f}")

accuracy on test set is 0.564


# 3. Locality-Sensitive Hashing and document search

In this part of the assignment, you will implement a more efficient version
of k-nearest neighbors using locality sensitive hashing.
You will then apply this to document search.

* Process the tweets and represent each tweet as a vector (represent a
document with a vector embedding).
* Use locality sensitive hashing and k nearest neighbors to find tweets
that are similar to a given tweet.

In [125]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
all_tweets = all_positive_tweets + all_negative_tweets

### 3.1 Getting the document embeddings

#### Bag-of-words (BOW) document models
Text documents are sequences of words.
* The ordering of words makes a difference. For example, sentences "Apple pie is
better than pepperoni pizza." and "Pepperoni pizza is better than apple pie"
have opposite meanings due to the word ordering.
* However, for some applications, ignoring the order of words can allow
us to train an efficient and still effective model.
* This approach is called Bag-of-words document model.

#### Document embeddings
* Document embedding is created by summing up the embeddings of all words
in the document.
* If we don't know the embedding of some word, we can ignore that word.

In [126]:
len(en_embeddings_subset[list(en_embeddings_subset.keys())[0]])

300

In [127]:
def get_document_embedding(tweet, en_embeddings): 
    '''
    Input:
        - tweet: a string
        - en_embeddings: a dictionary of word embeddings
    Output:
        - doc_embedding: sum of all word embeddings in the tweet
    '''
    embedding_dim = len(
        en_embeddings[list(en_embeddings.keys())[0]])
    doc_embedding = np.zeros(embedding_dim)
    processed_doc = process_tweet(tweet)
    for word in processed_doc:
        doc_embedding += en_embeddings.get(word, np.zeros(embedding_dim))
    return doc_embedding

In [128]:
# testing your function
custom_tweet = ('RT @Twitter @chapagain Hello There! Have a great day. '
                ':) #good #morning http://chapagain.com.np')
tweet_embedding = get_document_embedding(
    custom_tweet, en_embeddings_subset)
tweet_embedding[-5:]

array([-0.00268555, -0.15378189, -0.55761719, -0.07216644, -0.32263184])

#### Store all document vectors into a dictionary
Now, let's store all the tweet embeddings into a dictionary.
Implement `get_document_vecs()`

In [129]:
def get_document_vecs(all_docs, en_embeddings):
    '''
    Input:
        - all_docs: list of strings - all tweets in our dataset.
        - en_embeddings: dictionary with words as the keys and their 
          embeddings as the values.
    Output:
        - document_vec_matrix: matrix of tweet embeddings.
        - ind2Doc_dict: dictionary with indices of tweets in vecs as keys 
          and their embeddings as the values.
    '''
    # the dictionary's key is an index (integer) that identifies a 
    # specific tweet the value is the document embedding for that document
    ind2Doc_dict = {}
    doc_vecs = []

    for i, doc in enumerate(all_docs):
        doc_embedding = get_document_embedding(doc, en_embeddings)
        ind2Doc_dict[i] = doc_embedding
        doc_vecs.append(doc_embedding)
    # convert the list of document vectors into a 2D array (each row is a
    # document vector)
    #doc_vec_matrix = np.vstack(doc_vecs)
    doc_vec_matrix = np.array(doc_vecs)
    return doc_vec_matrix, ind2Doc_dict

In [130]:
document_vecs, ind2Tweet = get_document_vecs(all_tweets, 
                                             en_embeddings_subset)

In [131]:
print(f'length of dictionary {len(ind2Tweet)}')
print(f'shape of document_vecs {document_vecs.shape}')

length of dictionary 10000
shape of document_vecs (10000, 300)


## 3.2 Looking up the tweets

Now you have a vector of dimension (m,d) where `m` is the number of tweets
(10,000) and `d` is the dimension of the embeddings (300).  Now you
will input a tweet, and use cosine similarity to see which tweet in our
corpus is similar to your tweet.

In [132]:
my_tweet = 'stupid fruit'
process_tweet(my_tweet)
tweet_embedding = get_document_embedding(my_tweet, en_embeddings_subset)

In [133]:
idx = np.argmax(cosine_similarity(document_vecs, tweet_embedding))
print(all_tweets[idx])

I want to go to my grandma and grandpa's farm! There's a lot of cow chicken sheeps horses fruits and vegetables there :( Aww help me!


## 3.3 Finding the most similar tweets with LSH

You will now implement locality sensitive hashing (LSH) to identify the most similar tweet.
* Instead of looking at all 10,000 vectors, you can just search a subset to find
its nearest neighbors.

Let's say your data points are plotted in a vector spoace like this:

```
      ________________
     |                |
     |                |
     |                |
     |                |
     |                |
     |                |
     |                |
     |________________|
     
```

You can divide the vector space into regions and search within one region for nearest neighbors of a given vector.

```
      ________________
     |       |        |
     |      /|        |
     |     / |        |
     |    /  |________|
     |   /   |        |
     |  /    |        |
     | /     |        |
     |/______|________|
```

In [134]:
N_VECS = len(all_tweets)
N_DIMS = len(ind2Tweet[1])
print(f"Number of vectors is {N_VECS} and each has {N_DIMS} dimensions.")

Number of vectors is 10000 and each has 300 dimensions.


In [135]:
# The number of planes. We use log2(256) to have ~16 vectors/bucket.
N_PLANES = 10
# Number of times to repeat the hashing to improve the search.
N_UNIVERSES = 25

## 3.4 Getting the hash number for a vector

For each vector, we need to get a unique number associated to that vector in order to assign it to a "hash bucket".

### Hyperlanes in vector spaces
* In $3$-dimensional vector space, the hyperplane is a regular plane. In $2$ dimensional vector space, the hyperplane is a line.
* Generally, the hyperplane is subspace which has dimension $1$ lower than the original vector space has.
* A hyperplane is uniquely defined by its normal vector.
* Normal vector $n$ of the plane $\pi$ is the vector to which all vectors in the plane $\pi$ are orthogonal (perpendicular in $3$ dimensional case).

### Using Hyperplanes to split the vector space
We can use a hyperplane to split the vector space into $2$ parts.
* All vectors whose dot product with a plane's normal vector is positive are on one side of the plane.
* All vectors whose dot product with the plane's normal vector is negative are on the other side of the plane.

### Encoding hash buckets
* For a vector, we can take its dot product with all the planes, then encode this information to assign the vector to a single hash bucket.
* When the vector is pointing to the opposite side of the hyperplane than normal, encode it by 0.
* Otherwise, if the vector is on the same side as the normal vector, encode it by 1.
* If you calculate the dot product with each plane in the same order for every vector, you've encoded each vector's unique hash ID as a binary number, like [0, 1, 1, ... 0].

### Implementing hash buckets

We've initialized hash table `hashes` for you. It is list of `N_UNIVERSES` matrices, each describes its own hash table. Each matrix has `N_DIMS` rows and `N_PLANES` columns. Every column of that matrix is a `N_DIMS`-dimensional normal vector for each of `N_PLANES` hyperplanes which are used for creating buckets of the particular hash table.

*Exercise*: Your task is to complete the function `hash_value_of_vector` which places vector `v` in the correct hash bucket.

* First multiply your vector `v`, with a corresponding plane. This will give you a vector of dimension $(1,\text{N_planes})$.
* You will then convert every element in that vector to 0 or 1.
* You create a hash vector by doing the following: if the element is negative, it becomes a 0, otherwise you change it to a 1.
* You then compute the unique number for the vector by iterating over `N_PLANES`
* Then you multiply $2^i$ times the corresponding bit (0 or 1).
* You will then store that sum in the variable `hash_value`.

**Intructions:** Create a hash for the vector in the function below.
Use this formula:

$$ hash = \sum_{i=0}^{N-1} \left( 2^{i} \times h_{i} \right) $$

#### Create the sets of planes
* Create multiple (25) sets of planes (the planes that divide up the region).
* You can think of these as 25 separate ways of dividing up the vector space with a different set of planes.
* Each element of this list contains a matrix with 300 rows (the word vector have 300 dimensions), and 10 columns (there are 10 planes in each "universe").

In [136]:
np.random.seed(0)
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES))
            for _ in range(N_UNIVERSES)]

In [137]:
def hash_value_of_vector(v, planes):
    """Create a hash for a vector; hash_id says which random hash to use.
    Input:
        - v: vector of tweet. It's dimension is (1, N_DIMS)
        - planes: matrix of dimension (N_DIMS, N_PLANES) - the set of 
          planes that divide up the region
    Output:
        - res: a number which is used as a hash for your vector
    """
    # for the set of planes, calculate the dot product between the vector
    # and the matrix containing the planes. Remember that planes has shape
    # (300, 10). The dot product will have the shape (1,10)
    dot_product = np.dot(v, planes)
    sign = np.sign(dot_product)
    h = sign >= 0
    h = np.squeeze(h)
    hash_value = 0
    n_planes = planes.shape[1]
    for i in range(n_planes):
        # increment the hash value by 2^i * h_i
        hash_value += (2**i * h[i])
    hash_value = int(hash_value)
    return hash_value

In [138]:
np.random.seed(0)
idx = 0
planes = planes_l[idx]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
print(f" The hash value for this vector,",
      f"and the set of planes at index {idx},",
      f"is {hash_value_of_vector(vec, planes)}")

 The hash value for this vector, and the set of planes at index 0, is 768


## 3.5 Creating a hash table

Given that you have a unique number for each vector (or tweet), You now want to create a hash table. You need a hash table, so that given a hash_id, you can quickly look up the corresponding vectors. This allows you to reduce your search by a significant amount of time.

```
docs = [b0, b1, b2, ..., bn]
b0 = [tweet1, tweet8, tweet46, ...]
```

We have given you the `make_hash_table` function, which maps the tweet vectors to a bucket and stores the vector there. It returns the `hash_table` and the `id_table`. The `id_table` allows you know which vector in a certain bucket corresponds to what tweet.

In [139]:
def make_hash_table(vecs, planes):
    """
    Input:
        - vecs: list of vectors to be hashed.
        - planes: the matrix of planes in a single "universe", with shape
          (embedding dimensions, number of planes).
    Output:
        - hash_table: dictionary - keys are hashes, values are lists of 
          vectors (hash buckets)
        - id_table: dictionary - keys are hashes, values are list of 
          vectors id's (it's used to know which tweet corresponds to the 
          hashed vector)
    """
    n_planes = planes.shape[1]
    n_buckets = 2 ** n_planes
    hash_table = {n: [] for n in range(num_buckets)}
    id_table = {n: [] for n in range(num_buckets)}
    for i, v in enumerate(vecs):
        h = hash_value_of_vector(v, planes)
        hash_table[h].append(v)
        id_table[h].append(i)
    return hash_table, id_table

In [140]:
np.random.seed(0)
planes = planes_l[0]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
tmp_hash_table, tmp_id_table = make_hash_table(document_vecs, planes)

print(f'The hash table at key 0 has {len(tmp_hash_table[0])} '
      'document vectors')
print(f'The id table at key 0 has {len(tmp_id_table[0])}')
print(f'The first 5 document indices stored at key 0 of are '
      f'{tmp_id_table[0][0:5]}')

The hash table at key 0 has 3 document vectors
The id table at key 0 has 3
The first 5 document indices stored at key 0 of are [3276, 3281, 3282]


### 3.6 Creating all hash tables

You can now hash your vectors and store them in a hash table that
would allow you to quickly look up and search for similar vectors.
Run the cell below to create the hashes. By doing so, you end up having
several tables which have all the vectors. Given a vector, you then
identify the buckets in all the tables.  You can then iterate over the
buckets and consider much fewer vectors. The more buckets you use, the
more accurate your lookup will be, but also the longer it will take.

In [141]:
# Creating the hashtables
hash_tables = []
id_tables = []
for universe_id in range(N_UNIVERSES):  # there are 25 hashes
    print('working on hash universe #:', universe_id)
    planes = planes_l[universe_id]
    hash_table, id_table = make_hash_table(document_vecs, planes)
    hash_tables.append(hash_table)
    id_tables.append(id_table)

working on hash universe #: 0
working on hash universe #: 1
working on hash universe #: 2
working on hash universe #: 3
working on hash universe #: 4
working on hash universe #: 5
working on hash universe #: 6
working on hash universe #: 7
working on hash universe #: 8
working on hash universe #: 9
working on hash universe #: 10
working on hash universe #: 11
working on hash universe #: 12
working on hash universe #: 13
working on hash universe #: 14
working on hash universe #: 15
working on hash universe #: 16
working on hash universe #: 17
working on hash universe #: 18
working on hash universe #: 19
working on hash universe #: 20
working on hash universe #: 21
working on hash universe #: 22
working on hash universe #: 23
working on hash universe #: 24


### Approximate K-NN

Implement approximate K nearest neighbors using locality sensitive hashing,
to search for documents that are similar to a given document at the
index `doc_id`.

##### Inputs
* `doc_id` is the index into the document list `all_tweets`.
* `v` is the document vector for the tweet in `all_tweets` at index `doc_id`.
* `planes_l` is the list of planes (the global variable created earlier).
* `k` is the number of nearest neighbors to search for.
* `num_universes_to_use`: to save time, we can use fewer than the total
number of available universes.  By default, it's set to `N_UNIVERSES`,
which is $25$ for this assignment.

The `approximate_knn` function finds a subset of candidate vectors that
are in the same "hash bucket" as the input vector 'v'.  Then it performs
the usual k-nearest neighbors search on this subset (instead of searching
through all 10,000 tweets).

In [142]:
x = {1, 2, 3}
x.add(4)
x

{1, 2, 3, 4}

In [157]:
def approximate_knn(
        doc_id, v, planes_l, k=1, iters=N_UNIVERSES):
    '''Search for k-NN using hashes'''
    assert iters <= N_UNIVERSES
    candidate_vecs = []
    candidate_ids = []
    candidate_id_set = set()
    for universe_id in range(iters):
        planes = planes_l[universe_id]
        hash_value = hash_value_of_vector(v, planes)
        hash_table = hash_tables[universe_id]
        document_vectors_l = hash_table[hash_value]
        id_table = id_tables[universe_id]
        new_ids_to_consider = id_table[hash_value]
        if doc_id in new_ids_to_consider:
            new_ids_to_consider.remove(doc_id)
            print(f'removed doc_id {doc_id} of input vector from '
                  'new_ids_to_search')
        for i, new_id in enumerate(new_ids_to_consider):
            if new_id not in candidate_id_set:
                document_vector_at_i = document_vectors_l[i]
                candidate_vecs.append(document_vector_at_i)
                candidate_ids.append(new_id)
                candidate_id_set.add(new_id)
    print(f'There were {len(candidate_vecs)} vecs')
    candidate_vecs = np.array(candidate_vecs)
    nn_idxs = nearest_neighbor(v, candidate_vecs, k=k)
    nn_ids = [candidate_ids[idx] for idx in nn_idxs]
    return nearest_neighbor_ids

In [158]:
doc_id = 0
doc_to_search = all_tweets[doc_id]
vec_to_search = document_vecs[doc_id]

In [159]:
nearest_neighbor_ids = approximate_knn(
    doc_id, vec_to_search, planes_l, k=3, iters=5)

There were 77 vecs


In [160]:
print(f"Nearest neighbors for document {doc_id}")
print(f"Document contents: {doc_to_search}")
print("")

for neighbor_id in nearest_neighbor_ids:
    print(f"Nearest neighbor at document id {neighbor_id}")
    print(f"document contents: {all_tweets[neighbor_id]}")

Nearest neighbors for document 0
Document contents: #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

Nearest neighbor at document id 2140
document contents: @PopsRamjet come one, every now and then is not so bad :)
Nearest neighbor at document id 701
document contents: With the top cutie of Bohol :) https://t.co/Jh7F6U46UB
Nearest neighbor at document id 51
document contents: #FollowFriday @France_Espana @reglisse_menthe @CCI_inter for being top engaged members in my community this week :)
